In [ ]:
!pip install mido
!pip install music21
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=ed3ead0aaad5dab2e574b823e4da9f665dda0151c8fba4a049e7690626509bd3
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
import numpy as np
from mido import MidiFile
import IPython
import matplotlib.pyplot as plt
import librosa.display
import keras.layers as L
import keras.models as M
import keras
from keras.layers import SimpleRNN,LSTM,GRU
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout, LSTM, Conv2DTranspose, Conv2D, LeakyReLU, GlobalMaxPooling2D, Reshape, Flatten, BatchNormalization, Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
# from keras.layers.embeddings import Embedding
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import RMSprop, Adam

from sklearn.model_selection import train_test_split
from IPython import *
import os
import tensorflow as tf

from numpy.random import choice

from mido import Message, MidiFile, MidiTrack
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StructType, StructField, IntegerType
from mido import MidiFile
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StructType, StructField, IntegerType, StringType


In [ ]:
key_notes = {
    'Cb': 59,
    'C': 60,
    'C#': 61,
    'Db': 61,
    'D': 62,
    'D#': 63,
    'Eb': 63,
    'E': 64,
    'F': 65,
    'F#': 66,
    'Gb': 66,
    'G': 67,
    'G#': 68,
    'Ab': 68,
    'A': 69,
    'A#': 70,
    'Bb': 70,
    'B': 71
}

In [ ]:
spark = SparkSession.builder.getOrCreate()

# Read the CSV file
data = spark.read.csv('gs://cs649project/CS649Data/musicnet_metadata.csv', header=True, inferSchema=True)

# Display the first few rows of the DataFrame
data.show()

+----+--------+--------------------+--------------------+-------------+----------------+--------------------+------------+-------+
|  id|composer|         composition|            movement|     ensemble|          source|         transcriber|catalog_name|seconds|
+----+--------+--------------------+--------------------+-------------+----------------+--------------------+------------+-------+
|1727|Schubert|Piano Quintet in ...|          2. Andante|Piano Quintet|European Archive|http://tirolmusic...|       OP114|    447|
|1728|Schubert|Piano Quintet in ...|  3. Scherzo: Presto|Piano Quintet|European Archive|http://tirolmusic...|       OP114|    251|
|1729|Schubert|Piano Quintet in ...|4. Andantino - Al...|Piano Quintet|European Archive|http://tirolmusic...|       OP114|    444|
|1730|Schubert|Piano Quintet in ...|   5. Allegro giusto|Piano Quintet|European Archive|http://tirolmusic...|       OP114|    368|
|1733|Schubert|Piano Sonata in A...|        2. Andantino|   Solo Piano|        Muse

In [ ]:
# Define the path to the MIDI files directory
midi_files_dir = 'gs://cs649project/CS649Data/musicnet_midis/musicnet_midis/Beethoven/'

beethoven_midi_tracks = {}
n = 100
name = None

for m in range(n):
    # Read the MIDI file
    midi_path = os.path.join(midi_files_dir, os.listdir(midi_files_dir)[m])
    mid = MidiFile(midi_path, clip=True)

    # Process each track in the MIDI file
    for j in range(len(mid.tracks)):
        if j == 0:
            name = mid.tracks[j].name + ': '
        else:
            beethoven_midi_tracks[name + mid.tracks[j].name] = mid.tracks[j]

# Convert the dictionary to a DataFrame
beethoven_midi_tracks_df = spark.createDataFrame(list(beethoven_midi_tracks.items()), ['track_name', 'track_data'])

# Show the DataFrame
beethoven_midi_tracks_df.show()

In [ ]:
def get_key(s):
    k = None
    if 'key' in s:
        k = s[33:35]
        if k[-1] == "m" or k[-1] == "'":
            k = k[:-1]
    return k

def parse_notes(track):
    key = 'C'
    tunes = []
    new_tune = []
    note_dict = {}
    for i in track:
        if i.is_meta:
            new_key = get_key(str(i))
            if new_key is not None:
                key = new_key
            if len(tunes) > 0:
                tunes.append(new_tune)
                new_tune = []
        elif i.type == 'note_on' or i.type == 'note_off':
            if i.type == 'note_on' and i.dict()['velocity'] > 0 and i.dict()['time'] > 0:
                note_dict['time'] = i.dict()['time']
                note_dict['note'] = i.dict()['note']
                note_dict['velocity'] = i.dict()['velocity']
                note_dict['channel'] = i.dict()['channel']
            elif i.type == 'note_off' or i.type == 'note_on' and i.dict()['velocity'] == 0:
                if note_dict:
                    note_dict['pause'] = i.dict()['time']
                    note_dict['key'] = key
                    new_tune.append(note_dict)
                    note_dict = {}
    tunes.append(new_tune)
    return tunes

In [ ]:
# Define the path to the MIDI file
midi_file_path = 'gs://cs649project/CS649Data/musicnet_midis/musicnet_midis/Beethoven/'

# Read the MIDI file and get the tracks
midi = MidiFile(midi_file_path)
tracks = midi.tracks

# Convert the parse_notes function to a UDF
parse_notes_udf = udf(parse_notes, ArrayType(ArrayType(StructType([
    StructField("time", IntegerType()),
    StructField("note", IntegerType()),
    StructField("velocity", IntegerType()),
    StructField("channel", IntegerType()),
    StructField("pause", IntegerType()),
    StructField("key", StringType())
]))))

# Process each track and create a DataFrame
track_data = []
for track in tracks:
    track_data.append(parse_notes(track))
track_df = spark.createDataFrame(track_data, StringType())

# Show the track DataFrame
track_df.show(truncate=False)

In [ ]:
def various(notes):
    flag = True
    for i in range(8, len(notes)):
        flag = len(set(notes[i-8:i])) > 2
        if not flag:
            break
    return flag

In [ ]:
various_udf = udf(various, IntegerType())

# Convert tunes to a Spark DataFrame
tunes_df = spark.createDataFrame(tunes)

# Apply the parse_notes function to the tracks
tunes_df = tunes_df.selectExpr("parse_notes(track) as notes")

# Filter and create X and y DataFrames
X_df = tunes_df.filter(various_udf("notes")).selectExpr("explode(notes) as note").select("note.*")
y_df = X_df.selectExpr("lead(time) over (order by time) as next_time", "lead(note) over (order by time) as next_note", "lead(velocity) over (order by time) as next_velocity")

# Convert X and y DataFrames to numpy arrays
X = np.array(X_df.collect())
y = np.array(y_df.collect())

# Convert X and y arrays to integers
X = X.astype(int)
y = y.astype(int)

In [ ]:
X_rdd = sc.parallelize(X)
y_rdd = sc.parallelize(y)

#Store extracted features into a pickel file
import pickle


# Store data (serialize)
with open('X_rdd.pickle', 'wb') as handle:
    pickle.dump(X_rdd, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('y_rdd.pickle', 'wb') as handle:
    pickle.dump(y_rdd, handle, protocol=pickle.HIGHEST_PROTOCOL)

#The code loops over the tunes and extracts phrases of
# length defined by the ”phrase len” variable (in this case, 60).
# It checks the diversity of notes within the phrase using the
# ‘various(notes)‘ function. If the diversity condition is met,
# the note, duration, and velocity data for the phrase are ex-
# tracted and added to the ‘X‘ list, while the next note infor-
# mation is added to the ‘y‘ list